<a href="https://colab.research.google.com/github/mohanrajmit/DL-Training/blob/master/expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/FER.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/sad/Training_65242339.jpg  
  inflating: train/sad/Training_65267116.jpg  
  inflating: train/sad/Training_65275626.jpg  
  inflating: train/sad/Training_6529266.jpg  
  inflating: train/sad/Training_65329617.jpg  
  inflating: train/sad/Training_65338712.jpg  
  inflating: train/sad/Training_65338797.jpg  
  inflating: train/sad/Training_65387162.jpg  
  inflating: train/sad/Training_65404494.jpg  
  inflating: train/sad/Training_65426218.jpg  
  inflating: train/sad/Training_65430136.jpg  
  inflating: train/sad/Training_65437377.jpg  
  inflating: train/sad/Training_6545735.jpg  
  inflating: train/sad/Training_65463385.jpg  
  inflating: train/sad/Training_65473985.jpg  
  inflating: train/sad/Training_65502829.jpg  
  inflating: train/sad/Training_65505359.jpg  
  inflating: train/sad/Training_65508578.jpg  
  inflating: train/sad/Training_65516023.jpg  
  inflating: train/sad/Training_65524027.jpg  
  inflating

In [5]:
from imutils import paths
import cv2

In [6]:
imagePaths = list(paths.list_images("/content/train"))
print(imagePaths)

['/content/train/happy/Training_86546545.jpg', '/content/train/happy/Training_95177911.jpg', '/content/train/happy/Training_85248220.jpg', '/content/train/happy/Training_50979379.jpg', '/content/train/happy/Training_54862917.jpg', '/content/train/happy/Training_72651833.jpg', '/content/train/happy/Training_40559352.jpg', '/content/train/happy/Training_65407995.jpg', '/content/train/happy/Training_25462831.jpg', '/content/train/happy/Training_63501791.jpg', '/content/train/happy/Training_56213442.jpg', '/content/train/happy/Training_36648707.jpg', '/content/train/happy/Training_12109296.jpg', '/content/train/happy/Training_17425729.jpg', '/content/train/happy/Training_35877462.jpg', '/content/train/happy/Training_8874408.jpg', '/content/train/happy/Training_83974084.jpg', '/content/train/happy/Training_47490067.jpg', '/content/train/happy/Training_1968260.jpg', '/content/train/happy/Training_45601226.jpg', '/content/train/happy/Training_1954654.jpg', '/content/train/happy/Training_56315

In [8]:
def load(imagePaths):
  # initialize the list of features and labels
  data = []
  labels = []

  # loop over the input images
  for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label assuming
    # that our path has the following format:
    # /path/to/dataset/{class}/{image}.jpg
    image = cv2.imread(imagePath)
    label = imagePath.split("/")[-2]
    labels.append(label)
    #label = imagePath.split(os.path.sep)[-2]
    image = cv2.resize(image, (48, 48),interpolation=cv2.INTER_AREA)

    # treat our processed image as a "feature vector"
    # by updating the data list followed by the labels
    data.append(image)
    #labels.append(label)

    # show an update every `verbose` images
    #if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
    #print("[INFO] processed {}/{}".format(i + 1,
    #len(imagePaths)))

    # return a tuple of the data and labels
  return (np.array(data), np.array(labels))

In [9]:
print(len(imagePaths))

28709


In [10]:
(data, labels) = load(imagePaths)

In [11]:
print(data.shape)

(28709, 48, 48, 3)


In [12]:
print(labels.shape)

(28709,)


In [13]:
print(labels[0])

happy


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le_labels = le.fit_transform(labels)

In [16]:
le_labels

array([3, 3, 3, ..., 0, 0, 0])

In [17]:
le_labels.shape

(28709,)

In [18]:
y_train = to_categorical(le_labels)


In [20]:
labels[0]

'happy'

In [21]:
le_labels[0]

3

In [19]:
y_train[0]

array([0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [22]:

data = data.astype('float32')

data/=255

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten

In [24]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(48,48,3),padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(7,activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                   

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(data, y_train, batch_size=128, epochs=100)

Epoch 1/100
225/225 [==============================] - 19s 20ms/step - loss: 1.6473 - accuracy: 0.3490
Epoch 2/100
225/225 [==============================] - 4s 20ms/step - loss: 1.3621 - accuracy: 0.4779
Epoch 3/100
225/225 [==============================] - 4s 19ms/step - loss: 1.1959 - accuracy: 0.5478
Epoch 4/100
225/225 [==============================] - 4s 18ms/step - loss: 1.0838 - accuracy: 0.5920
Epoch 5/100
225/225 [==============================] - 5s 20ms/step - loss: 0.9664 - accuracy: 0.6404
Epoch 6/100
225/225 [==============================] - 4s 19ms/step - loss: 0.8519 - accuracy: 0.6848
Epoch 7/100
225/225 [==============================] - 4s 18ms/step - loss: 0.7325 - accuracy: 0.7305
Epoch 8/100
225/225 [==============================] - 4s 19ms/step - loss: 0.6011 - accuracy: 0.7830
Epoch 9/100
225/225 [==============================] - 4s 19ms/step - loss: 0.4776 - accuracy: 0.8278
Epoch 10/100
225/225 [==============================] - 4s 19ms/step - loss: 0.36